In [4]:
from ipywidgets import Dropdown, RadioButtons, VBox, Output
from IPython.display import display

import pandas as pd
import cobra
from fba4l import media
from escher import Builder
from bokeh.io import output_notebook, show, push_notebook
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.plotting import figure
import math
output_notebook()

# ---------------------------------------
# Simulation: Run model for each media under both conditions
# ---------------------------------------
fluxes_with = {}
fluxes_without = {}
summary = []

model = cobra.io.load_json_model('/Users/coltonlloyd/fba4l/fba4l/models/core_model.json')
model.reactions.EX_glc__D_c.knock_out()

for m in media:
    if m in model.reactions:
        # With PCCA:
        with model:
            model.reactions.get_by_id(m).lower_bound = -1
            sol = model.optimize()
            fluxes_with[m] = sol.fluxes.copy()
            summary.append(dict(media=m, atp=sol.objective_value, kind='with PCCA'))
        # Without PCCA:
        with model:
            model.reactions.get_by_id(m).lower_bound = -1
            model.reactions.PPCOACm.knock_out()
            sol = model.optimize()
            fluxes_without[m] = sol.fluxes.copy()
            summary.append(dict(media=m, atp=sol.objective_value, kind='without PCCA'))

# Create a DataFrame summary for the bar plot
df = pd.DataFrame(summary).sort_values('media')

# ---------------------------------------
# Bokeh Bar Plot: Show ATP yield for a chosen condition
# ---------------------------------------
# Start with one default condition (e.g., "with PCCA")
initial_condition = 'with PCCA'
df_condition = df[df['kind'] == initial_condition].copy()
# Instead of a tuple, define factor as just the media name.
df_condition['factor'] = df_condition['media']

source = ColumnDataSource(df_condition)
factors = list(df_condition['factor'])

p = figure(x_range=FactorRange(*factors), height=400, width=800,
           title="ATP Yield (%s)" % initial_condition, toolbar_location=None, tools="")

# For a single condition, use a fixed color.
p.vbar(x='factor', top='atp', width=0.8, source=source, line_color="white",
       fill_color="#718dbf")

# Rotate x-axis labels by 45 degrees
p.xaxis.major_label_orientation = math.radians(45)
p.xgrid.grid_line_color = None

bokeh_handle = show(p, notebook_handle=True)

# ---------------------------------------
# Escher Map: Create an Escher Builder for flux visualization
# ---------------------------------------
default_media = list(fluxes_with.keys())[0]
builder = Builder(map_json='/Users/coltonlloyd/fba4l/fba4l/maps/core_map_v30.json', 
                  reaction_data=fluxes_with[default_media])

builder.reaction_scale = [
    {'type': 'value', 'color': 'red', 'size': 20, 'value': -0.01},
    {'type': 'value', 'color': 'grey', 'size': 0, 'value': 0},
    {'type': 'value', 'color': 'red', 'size': 20, 'value': 0.01}
]

escher_out = Output()
with escher_out:
    display(builder)

# ---------------------------------------
# ipywidgets for interactive toggling
# ---------------------------------------
condition_widget = RadioButtons(options=['with PCCA', 'without PCCA'],
                                value=initial_condition, description='Condition:')
media_widget = Dropdown(options=sorted(list(fluxes_with.keys())),
                        value=default_media, description='Media:')

def update_outputs(change):
    # Get current selections
    cond = condition_widget.value
    med = media_widget.value

    # --- Update Bokeh bar plot ---
    df_new = df[df['kind'] == cond].copy()
    # Use only the media name as factor.
    df_new['factor'] = df_new['media']
    new_factors = list(df_new['factor'])
    source.data = df_new.to_dict(orient='list')
    p.x_range.factors = new_factors
    p.title.text = "ATP Yield (%s)" % cond

    # --- Update Escher map ---
    flux_data = fluxes_with[med] if cond == 'with PCCA' else fluxes_without[med]
    builder.reaction_data = flux_data

    push_notebook(handle=bokeh_handle)

# Attach the update function to both widgets
condition_widget.observe(update_outputs, names='value')
media_widget.observe(update_outputs, names='value')

# ---------------------------------------
# Display all interactive elements in the notebook
# ---------------------------------------
ui = VBox([condition_widget, media_widget, escher_out])
display(ui)

Loading BokehJS ...